In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import ipywidgets as wg
from IPython.display import display, clear_output
from ipywidgets import interact, interact_manual
import plotly.express as px

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot

import cufflinks as cf
cf.go_offline()

In [80]:
df = pd.read_csv("analysis.csv")

In [3]:
# b = df.memory_usage(index=True, deep=True).sum()
print('Dataframe loaded.') 

import sys
b = sys.getsizeof(df)

Dataframe loaded.


In [4]:
print('Processed ' + str(len(df)) + ' entries in table.')

import math
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

print('Mem cache: ' + str(convert_size(b))) 

Processed 75 entries in table.
Mem cache: 27.51 KB


In [79]:
s=wg.Dropdown(options=['Beds', 'TotalBldg_GSF'], description='Search by: ')
display(s)

s1=wg.Dropdown(options=df['Cat_Major'].unique(), description='Tab: ')
display(s1)

s2 = wg.SelectMultiple(
    options=df['Cat_Minor'].unique(),
    rows=10,
    description='Department: ',
    disabled=False
)

display(s2)

s3 = wg.FloatText(
    max='10000',
    step='100',
    description='Query: '
)
display(s3)

# display(nn,pp)
# mylink = wg.jslink((nn,'value'),(pp,'value'))
# pp.max='10000'
# pp2.step='100'

out = wg.Output()
display(out)

def response(change):
    with out:
        clear_output(wait=True)
        if s.value == 'Beds':
            xT = 'Number of Beds'
        else:
            xT = 'Area'

        df2 = df[s.value].value_counts()
        df2 = df2.to_frame()
        df3 = df2.reset_index()
        # df3['index'].iplot(kind="histogram", bins=20, title="Number of projects undertaken", xTitle=xT, yTitle='Count', theme='white', color='blue')
        fig = px.histogram(df3, x="index", nbins=20, template='none', color_discrete_sequence=px.colors.sequential.RdBu,
                        labels={
                             "index": xT
                         },
                        title="Number of projects undertaken till date")
        fig.show()
        
        df4 = df.loc[df['Cat_Minor'].isin(s2.value)]
        # df4 = df.loc[df['Cat_Minor'].isin(s2.value) & df['Project'].isin(s3.value)]
        # If you also want to filter out projects, this shows mean now
        fig = px.pie(df4, values='Total_SF', names='Cat_Minor', title='Area Allocation (Mean)', hole=.5, color_discrete_sequence=px.colors.sequential.RdBu)
        fig.show()
        df5 = df4[['Cat_Minor', 'Total_SF']]
        df6 = df5.groupby(['Cat_Minor'], as_index=False).sum()
        df6 = df6.style.set_properties(**{'text-align': 'left'})

s.observe(response, names="value")
s1.observe(response, names="value")
s2.observe(response, names="value")
s3.observe(response, names="value")

Dropdown(description='Search by: ', options=('Beds', 'TotalBldg_GSF'), value='Beds')

Dropdown(description='Tab: ', options=('D&T', 'OutPatient'), value='D&T')

SelectMultiple(description='Department: ', options=('Radiology', 'Surgery', 'Endoscopy', 'Emergency Dept.', 'L…

FloatText(value=0.0, description='Query: ', step=100.0)

Output()